In [ ]:
import functions_framework
import os

# NOTE: All other imports have been moved inside the function to ensure
# the fastest possible startup time and pass the health check.

@functions_framework.http
def trigger_pipeline(request):
    """
    HTTP-triggered function that publishes a message to a Pub/Sub topic
    to start the data processing pipeline using the REST API.
    """
    # Lazy import all required libraries to speed up function initialization
    import requests
    import google.auth
    import base64
    import json

    # --- CONFIGURATION ---
    # Your GCP project ID, fetched from the environment variable set during deployment
    PROJECT_ID = os.environ.get('GCP_PROJECT', 'robotic-fuze-463700-n5')
    # The Pub/Sub topic to publish to
    TOPIC_ID = os.environ.get('PUBSUB_TOPIC', 'nightly-data-pull-topic')

    try:
        # Get the default credentials from the environment
        credentials, project = google.auth.default(
            scopes=["https://www.googleapis.com/auth/pubsub"]
        )

        # We need to refresh the credentials to get an access token
        auth_req = google.auth.transport.requests.Request()
        credentials.refresh(auth_req)

        # This is the token we'll use to authenticate our API request
        token = credentials.token

        # Construct the URL for the Pub/Sub REST API
        url = f"https://pubsub.googleapis.com/v1/projects/{PROJECT_ID}/topics/{TOPIC_ID}:publish"

        # The message must be base64-encoded
        message_data = b'Start data pipeline'
        base64_encoded_message = base64.b64encode(message_data).decode('utf-8')

        # Construct the request body
        payload = {
            "messages": [
                {
                    "data": base64_encoded_message
                }
            ]
        }

        # Set the authorization header
        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json; charset=utf-8"
        }

        # Make the POST request to the API
        response = requests.post(url, data=json.dumps(payload), headers=headers)
        response.raise_for_status() # This will raise an exception for bad status codes

        message_ids = response.json().get('messageIds', [])
        print(f"Published message(s) {message_ids} to {TOPIC_ID}.")

        return f"Successfully triggered pipeline (Message IDs: {message_ids}).", 200

    except Exception as e:
        print(f"Error publishing to Pub/Sub via REST API: {e}")
        # If the response exists, print its content for more details
        if 'response' in locals() and hasattr(response, 'text'):
            print(f"Response body: {response.text}")
        return "Error triggering pipeline.", 500
